In [1]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [2]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [3]:
# Load csvs from raw_data folder
trends1 = pd.read_csv('../processed_data/LAB_CON_LIB_GRE_BRX_Trends_2004_now.csv', skiprows=1)
trends2 = pd.read_csv('../processed_data/LAB_PLC_SNP_UKI_NAT_Trends_2004_now.csv', skiprows=1)

In [4]:
# Drop duplicate Labour column
trends2.drop(columns='Labour Party: (United Kingdom)',inplace=True)

In [5]:
# Join two trends datasets on 'Month'
trends_merged = pd.merge(trends1,trends2,how='left',on='Month')

In [6]:
# Convert 'Month' field to pd.datetime64[ns] format for joining with polling data
trends_merged['Month'] = pd.to_datetime(trends_merged['Month'])

In [7]:
# Replace <1 values with 0.5 value
trends_merged.replace('<1', 0.5,inplace=True)

In [8]:
# Converting objects (pd.Series) to int dtypes
trends_merged = trends_merged.astype({'Green Party: (United Kingdom)':'int','Reform UK: (United Kingdom)':'int',\
    'Plaid Cymru: (United Kingdom)':'int','Scottish National Party: (United Kingdom)':'int',\
        'UK Independence Party: (United Kingdom)':'int','British National Party: (United Kingdom)':'int'})

In [9]:
# Rename columns to align them more easily with _FC and _ACT columns
trends_merged.rename(columns={'Labour Party: (United Kingdom)': 'LAB_trends',
        'Conservative Party: (United Kingdom)': 'CON_trends',
        'Liberal Democrats: (United Kingdom)': 'LIB_trends',
        'Green Party: (United Kingdom)': 'GRE_trends',
        'Reform UK: (United Kingdom)': 'BRX_trends',
        'Plaid Cymru: (United Kingdom)': 'PLC_trends',
        'Scottish National Party: (United Kingdom)': 'SNP_trends',
        'UK Independence Party: (United Kingdom)': 'UKI_trends',
        'British National Party: (United Kingdom)': 'NAT_trends'}
        ,inplace=True)

In [10]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [11]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [12]:
# Create poll enddate field with YYYY-MM format, so we can join with Trends data
data['enddate_year_month'] = pd.to_datetime(data['enddate']).dt.to_period('M')

In [13]:
# Convert Period datatype to str and then to datetime64
data.enddate_year_month = pd.to_datetime(data.enddate_year_month.astype('str'))

In [14]:
# Merge data with trends data, to get a df with polls, election results and trends
data = pd.merge(data,trends_merged,how='left',left_on='enddate_year_month',right_on='Month')

In [15]:
data.fillna(value=0,inplace=True)

/tmp/ipykernel_65019/3951631895.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  data.fillna(value=0,inplace=True)


In [16]:
# Scale trends columns by diving by 100
for column in ['LAB_trends', 'CON_trends', 'LIB_trends',
       'GRE_trends', 'BRX_trends', 'PLC_trends', 'SNP_trends', 'UKI_trends',
       'NAT_trends']:
    data[column] = data[column] / 100

In [17]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [18]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [19]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [20]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [21]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1397
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [22]:
# Define election dates
election_date = datetime.strptime('2024-07-04', '%Y-%m-%d')
cutoff_date = election_date - timedelta(days=54)
prediction_date = election_date - timedelta(days=24)

In [23]:
# Split data
data_train = data[data['startdate'] > '2003-12-31']
data_train = data_train[data_train['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date]

In [56]:
data_test

,Unnamed: 0,startdate,enddate,pollster,samplesize,rating,next_elec_date,days_to_elec,months_to_elec,months_to_elec_weight,...,Month,LAB_trends,CON_trends,LIB_trends,GRE_trends,BRX_trends,PLC_trends,SNP_trends,UKI_trends,NAT_trends
5649,5649,2024-05-12,2024-05-12,RedfieldWilton,3000,D,2024-07-04,53,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5654,5654,2024-05-16,2024-05-16,PeoplePolling,1476,D,2024-07-04,49,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5655,5655,2024-05-15,2024-05-16,Techne,1641,D,2024-07-04,50,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5656,5656,2024-05-15,2024-05-16,YouGov,1031,A-,2024-07-04,50,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5657,5657,2024-05-16,2024-05-17,Omnisis,1064,D,2024-07-04,49,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5658,5658,2024-05-15,2024-05-17,Opinium,1458,A-,2024-07-04,50,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5659,5659,2024-05-17,2024-05-19,MoreinCommon,2308,F,2024-07-04,48,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5660,5660,2024-05-19,2024-05-19,RedfieldWilton,3700,D,2024-07-04,46,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5661,5661,2024-05-17,2024-05-19,SavantaComRes,2295,B+,2024-07-04,48,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0
5662,5662,2024-05-17,2024-05-20,Deltapoll,1968,D+,2024-07-04,48,2,0.732755,...,2024-05-01 00:00:00,0.31,0.10,0.07,0.03,0.09,0.00,0.04,0.01,0.0


In [24]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)

In [25]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [26]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [27]:
# Define our X by dropping irrelevant and y columns
X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [28]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [29]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2024-07-04',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [30]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                                                  0.033863
months_to_elec_weight                                       0.772587
rating                                                      0.634615
party_in_power_Conservative                                      1.0
party_in_power_Conservative_Liberal                              0.0
poll_length                                                 2.134615
CON_FC                                                      0.230577
LAB_FC                                                      0.445962
LIB_FC                                                      0.094038
BRX_FC                                                      0.120577
GRE_FC                                                      0.057885
OTH_FC                                                      0.018654
PLC_FC                                                      0.004423
SNP_FC                                                      0.026538
UKI_FC                            

In [31]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': averages['BRX_FC'],
    'GRE_ACT': averages['GRE_FC'],
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT': averages['PLC_FC'],
    'SNP_ACT': averages['SNP_FC'],
    'UKI_ACT': averages['UKI_FC']
}

In [32]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


/tmp/ipykernel_65019/3014033841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.fillna(value=imputation_values)


In [33]:
X_train['UKI_FC'] = 0
X_test['UKI_FC'] = 0

In [34]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [35]:
X_train.columns

Index(['samplesize', 'months_to_elec_weight', 'rating',
       'party_in_power_Conservative', 'party_in_power_Conservative_Liberal',
       'poll_length', 'CON_FC', 'LAB_FC', 'LIB_FC', 'BRX_FC', 'GRE_FC',
       'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC', 'enddate_year_month', 'Month',
       'LAB_trends', 'CON_trends', 'LIB_trends', 'GRE_trends', 'BRX_trends',
       'PLC_trends', 'SNP_trends', 'UKI_trends', 'NAT_trends'],
      dtype='object')

In [36]:
X_train.drop(columns='enddate_year_month',inplace=True)
X_train.drop(columns='Month',inplace=True)

In [37]:
X_test.drop(columns='enddate_year_month',inplace=True)
X_test.drop(columns='Month',inplace=True)

In [38]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [39]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [40]:
# Print mean predictions
for party in parties:
    print(f"2024 mean prediction for {party}: {mean_predictions[party]}")


2024 mean prediction for CON: 0.2768957018852234
2024 mean prediction for LAB: 0.3929210901260376
2024 mean prediction for LIB: 0.09599817544221878
2024 mean prediction for BRX: 0.10434237867593765
2024 mean prediction for GRE: 0.05288068950176239
2024 mean prediction for SNP: 0.028291622176766396
2024 mean prediction for UKI: -0.001720459433272481
2024 mean prediction for PLC: 0.00457283528521657
2024 mean prediction for OTH: 0.025849970057606697


In [51]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-4]

In [52]:
# Print mean of predictions
for party in parties:
    print(f"2024 last prediction for {party}: {last_predictions[party].mean()}")


2024 last prediction for CON: 0.3369247019290924
2024 last prediction for LAB: 0.3110770285129547
2024 last prediction for LIB: 0.09830141812562943
2024 last prediction for BRX: 0.0821041464805603
2024 last prediction for GRE: 0.046387311071157455
2024 last prediction for SNP: 0.031485654413700104
2024 last prediction for UKI: -0.004656370263546705
2024 last prediction for PLC: 0.004716698545962572
2024 last prediction for OTH: 0.03573805093765259
